# Déploiement du Modèle SSD MobileNet V2

Ce notebook automatise le déploiement du modèle SSD MobileNet V2 entraîné vers le système de versioning de l'API.

## Processus de déploiement

1. **Analyse du modèle** : Vérification du modèle SavedModel
2. **Versioning automatique** : Incrémentation intelligente de la version
3. **Copie et métadonnées** : Déploiement avec informations complètes
4. **Validation** : Test du modèle déployé

In [4]:
import os
import shutil
import json
import tensorflow as tf
from datetime import datetime
from pathlib import Path
import re

# Configuration des chemins
TRAINING_MODEL_PATH = "/home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/exported_model/saved_model"
API_VERSIONS_PATH = "/home/sarsator/projets/gaia_vision/api/models/dl_model/versions"
TRAINING_NOTEBOOK_PATH = "/home/sarsator/projets/gaia_vision/training/notebook/dl_finetuning.ipynb"

print("Configuration deployment")
print(f"Source model: {TRAINING_MODEL_PATH}")
print(f"Destination API: {API_VERSIONS_PATH}")

# Check et creation du model path
if not os.path.exists(TRAINING_MODEL_PATH):
    print("Dossier model source non trouve")
    os.makedirs(TRAINING_MODEL_PATH, exist_ok=True)
    print("Dossier model cree")
    print("Note: Model doit etre exporte avant deployment")
else:
    print("Dossier model source OK")

# Check et creation du versions path  
if not os.path.exists(API_VERSIONS_PATH):
    os.makedirs(API_VERSIONS_PATH, exist_ok=True)
    print("Dossier versions cree")
else:
    print("Dossier versions OK")

print("Paths valides")

Configuration deployment
Source model: /home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/exported_model/saved_model
Destination API: /home/sarsator/projets/gaia_vision/api/models/dl_model/versions
Dossier model source OK
Dossier versions OK
Paths valides


In [5]:
def get_next_version():
    """Analyse versions existantes et retourne la prochaine version"""
    
    if not os.path.exists(API_VERSIONS_PATH):
        return "1.0", "20250716_000000"
    
    # Lister versions existantes
    versions = []
    for folder in os.listdir(API_VERSIONS_PATH):
        if os.path.isdir(os.path.join(API_VERSIONS_PATH, folder)):
            # Format: v1.2_20250711_143350
            match = re.match(r'v(\d+)\.(\d+)_(\d{8}_\d{6})', folder)
            if match:
                major, minor, timestamp = match.groups()
                versions.append((int(major), int(minor), timestamp, folder))
    
    if not versions:
        return "1.0", "20250716_000000"
    
    # Trier par version
    versions.sort(key=lambda x: (x[0], x[1]))
    latest_major, latest_minor, _, latest_folder = versions[-1]
    
    print(f"Latest version: v{latest_major}.{latest_minor}")
    
    # Incrementer version mineure
    next_version = f"{latest_major}.{latest_minor + 1}"
    next_timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    print(f"Next version: v{next_version}_{next_timestamp}")
    
    return next_version, next_timestamp

# Get next version
next_version, next_timestamp = get_next_version()

Latest version: v1.7
Next version: v1.8_20250722_182730


In [6]:
# Analyser le modèle actuel
def analyze_model():
    """Analyse du modèle actuel"""
    
    print("=== ANALYSE DU MODELE ===")
    
    # Vérifier l'existence des fichiers
    pb_path = os.path.join(TRAINING_MODEL_PATH, "saved_model.pb")
    checkpoint_path = os.path.join(TRAINING_MODEL_PATH, "..", "..", "training_2025_07_19_5734", "ckpt-121")
    
    if os.path.exists(pb_path):
        size_mb = os.path.getsize(pb_path) / (1024 * 1024)
        print(f"Modèle Frozen Graph: {size_mb:.2f} MB")
    else:
        print("Modèle Frozen Graph: Non trouvé")
    
    # Analyser checkpoint
    if os.path.exists(checkpoint_path + ".index"):
        print(f"Checkpoint: {checkpoint_path}")
        print("Checkpoint disponible")
    else:
        print("Checkpoint: Non trouvé")
    
 
    
    return True

# Analyser le modèle
model_analysis = analyze_model()

=== ANALYSE DU MODELE ===
Modèle Frozen Graph: Non trouvé
Checkpoint: Non trouvé


## Export du model entrainer


In [8]:
# Export du checkpoint vers SavedModel
def export_trained_model():
    """Export le checkpoint trained en SavedModel"""
    
    # Paths config
    checkpoint_dir = "/home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320"
    pipeline_config = "/home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/pipeline_working.config"
    output_dir = "/home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/exported_model"
    
    print("\033[94mDemarrage export model\033[0m")
    print(f"Checkpoint dir: {checkpoint_dir}")
    print(f"Config file: {pipeline_config}")
    print(f"Export dir: {output_dir}")
    
    # Check si checkpoint existe
    if not os.path.exists(checkpoint_dir):
        print("\033[91mErreur: Checkpoint dir non trouve\033[0m")
        return False
    
    if not os.path.exists(pipeline_config):
        print("\033[91mErreur: Config file non trouve\033[0m")
        return False
    
    # Create export directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Import TF Object Detection API
    import sys
    import subprocess
    
    # Path vers le script exporter
    research_path = '/home/sarsator/projets/gaia_vision/training/notebook/tensorflow_models/research'
    exporter_script = os.path.join(research_path, 'object_detection', 'exporter_main_v2.py')
    
    if not os.path.exists(exporter_script):
        print(f"\033[91mErreur: Script exporter non trouve: {exporter_script}\033[0m")
        return False
    
    print("\033[92mScript exporter trouve\033[0m")
    
    # Export command via subprocess
    export_cmd = [
        sys.executable,
        exporter_script,
        '--input_type=image_tensor',
        f'--pipeline_config_path={pipeline_config}',
        f'--trained_checkpoint_dir={checkpoint_dir}',
        f'--output_directory={output_dir}'
    ]
    
    print("\033[93mLancement export...\033[0m")
    print("Export command:", ' '.join(export_cmd))
    
    try:
        # Setup environment
        env = os.environ.copy()
        env['PYTHONPATH'] = f"{research_path}:{research_path}/slim:{env.get('PYTHONPATH', '')}"
        
        # Run export via subprocess
        result = subprocess.run(
            export_cmd,
            env=env,
            cwd=research_path,
            capture_output=True,
            text=True,
            timeout=300  # 5 minutes timeout
        )
        
        if result.returncode == 0:
            print("\033[92mExport completed successfully\033[0m")
            
            # Verify export
            saved_model_path = os.path.join(output_dir, "saved_model")
            if os.path.exists(saved_model_path):
                print(f"\033[92mSavedModel available: {saved_model_path}\033[0m")
                return True
            else:
                print("\033[91mErreur: SavedModel non cree\033[0m")
                return False
        else:
            print(f"\033[91mErreur durant export (code {result.returncode})\033[0m")
            print(f"Stdout: {result.stdout}")
            print(f"Stderr: {result.stderr}")
            return False
            
    except subprocess.TimeoutExpired:
        print("\033[91mErreur: Export timeout (>5min)\033[0m")
        return False
    except Exception as e:
        print(f"\033[91mErreur durant export: {e}\033[0m")
        return False

# Execute export
export_success = export_trained_model()

Demarrage export model
Checkpoint dir: /home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320
Config file: /home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/pipeline_working.config
Export dir: /home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/exported_model
Script exporter trouve
Lancement export...
Export command: /home/sarsator/projets/gaia_vision/.venv/bin/python /home/sarsator/projets/gaia_vision/training/notebook/tensorflow_models/research/object_detection/exporter_main_v2.py --input_type=image_tensor --pipeline_config_path=/home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/pipeline_working.config --trained_checkpoint_dir=/home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320 --output_directory=/home/sarsator/projets/gaia_vision/training/models/dl_model/outputs/ssd_mnv2_320/exported_model
Export completed successfully
SavedModel available:

In [9]:
# Créer nouveau dossier de version
def create_new_version_folder(version, timestamp):
    """Créer dossier pour nouvelle version"""
    
    new_version_name = f"v{version}_{timestamp}"
    new_version_path = os.path.join(API_VERSIONS_PATH, new_version_name)
    
    print(f"Création: {new_version_name}")
    
    # Créer structure
    os.makedirs(new_version_path, exist_ok=True)
    
    # Copier modèle
    source_model = TRAINING_MODEL_PATH
    dest_model = os.path.join(new_version_path, "saved_model")
    
    if os.path.exists(source_model):
        shutil.copytree(source_model, dest_model)
        print(f"Modèle copié: {dest_model}")
    else:
        print("Erreur: Modèle source non trouvé")
        return None
    
    # Créer metadata
    metadata = {
        "version": version,
        "timestamp": timestamp,
        "model_type": "ssd_mobilenet_v2",
        "framework": "tensorflow",
        "metrics": {
            "mAP": 74.2,
            "mAP@0.50": 89.9,
            "mAP@0.75": 87.1,
            "recall": 77.7
        },
        "deployment_date": datetime.now().isoformat()
    }
    
    metadata_path = os.path.join(new_version_path, "metadata.json")
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"Version créée: {new_version_path}")
    return new_version_path

# Créer nouvelle version
new_version_path = create_new_version_folder(next_version, next_timestamp)

Création: v1.8_20250722_182730
Modèle copié: /home/sarsator/projets/gaia_vision/api/models/dl_model/versions/v1.8_20250722_182730/saved_model
Version créée: /home/sarsator/projets/gaia_vision/api/models/dl_model/versions/v1.8_20250722_182730


In [ ]:
# Mettre à jour symlink current (version optimisée)
def update_current_symlink(new_version_path):
    """Mettre à jour le symlink current vers la nouvelle version avec vérifications avancées"""
    
    print("=== MISE A JOUR SYMLINK CURRENT ===")
    
    versions_path = Path(API_VERSIONS_PATH)
    new_version_folder = Path(new_version_path)
    current_link = versions_path / "current"
    
    print(f"Nouvelle version: {new_version_folder.name}")
    
    # Vérifier que saved_model existe dans la nouvelle version
    saved_model_dir = new_version_folder / "saved_model"
    if not saved_model_dir.exists():
        print(f"ERREUR: saved_model non trouvé dans {new_version_folder}")
        return False
    
    print(f"SavedModel validé: {saved_model_dir}")
    
    # Supprimer ancien symlink s'il existe
    if current_link.exists() or current_link.is_symlink():
        old_target = current_link.resolve() if current_link.is_symlink() else None
        current_link.unlink()
        if old_target:
            print(f"Ancien symlink supprimé (pointait vers: {old_target.name})")
        else:
            print("Ancien fichier/dossier 'current' supprimé")
    
    # Créer nouveau symlink (relatif pour portabilité)
    current_link.symlink_to(new_version_folder.name)
    print(f"Nouveau symlink créé: current -> {new_version_folder.name}")
    
    # Vérification complète
    if current_link.exists() and current_link.is_symlink():
        target = current_link.resolve()
        if target == new_version_folder:
            print(f"✅ SUCCESS: Symlink current pointe correctement vers {new_version_folder.name}")
            
            # Test de validation du modèle via symlink
            test_saved_model = current_link / "saved_model"
            if test_saved_model.exists():
                print(f"✅ Validation: SavedModel accessible via symlink")
                return True
            else:
                print(f"ERREUR: SavedModel non accessible via symlink")
                return False
        else:
            print(f"ERREUR: Symlink pointe vers {target} au lieu de {new_version_folder}")
            return False
    else:
        print("ERREUR: Symlink non créé correctement")
        return False

# Mettre à jour symlink si version créée
if new_version_path:
    symlink_success = update_current_symlink(new_version_path)
    
    if symlink_success:
        print("\n🎉 DEPLOYMENT COMPLET ET SYMLINK MIS A JOUR!")
        print("Le modèle sera automatiquement utilisé par l'API")
    else:
        print("\n⚠️ Version créée mais symlink en erreur - intervention manuelle requise")
else:
    print("Erreur: Impossible de créer symlink - version non créée")

Ancien symlink supprimé
Nouveau symlink créé: current -> v1.8_20250722_182730
Symlink current mis à jour
